# European Soccer Database for data analysis Using SQLite,
- In this report, I will analyse the of reviews of fine foods from amazon. The data span a period of more than 10 years, including all ~500,000 reviews up to October 2012. Reviews include product and user information, ratings, and a plain text review. It also includes reviews from all other Amazon categories.

- Data includes:

  - Reviews from Oct 1999 - Oct 2012
  - 568,454 reviews
  - 256,059 users
  - 74,258 products
  - 260 users with > 50 reviews
  - wordcloud

https://www.kaggle.com/snap/amazon-fine-food-reviews?select=database.sqlite

In [1]:
import sqlite3
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
conn= sqlite3.connect('database.sqlite')

### Data Wrangeling

In [5]:
dataframe= pd.read_sql("""SELECT * FROM sqlite_master;""", conn)

In [6]:
dataframe

,type,name,tbl_name,rootpage,sql
0,table,Reviews,Reviews,2,CREATE TABLE Reviews (\n Id INTEGER PRIMARY...


In [7]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 5 columns):
type        1 non-null object
name        1 non-null object
tbl_name    1 non-null object
rootpage    1 non-null int64
sql         1 non-null object
dtypes: int64(1), object(4)
memory usage: 168.0+ bytes


In [8]:
dataframe.describe()

,rootpage
count,1.0
mean,2.0
std,NaN
min,2.0
25%,2.0
50%,2.0
75%,2.0
max,2.0


In [9]:
df_table= pd.read_sql("""SELECT * FROM Reviews;""", conn)

In [10]:
df_table

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...,...,...
568449,568450,B001EO7N10,A28KG5XORO54AY,Lettie D. Carter,0,0,5,1299628800,Will not do without,Great for sesame chicken..this is a good if no...
568450,568451,B003S1WTCU,A3I8AFVPEE8KI5,R. Sawyer,0,0,2,1331251200,disappointed,I'm disappointed with the flavor. The chocolat...
568451,568452,B004I613EE,A121AA1GQV751Z,"pksd ""pk_007""",2,2,5,1329782400,Perfect for our maltipoo,"These stars are small, so you can give 10-15 o..."
568452,568453,B004I613EE,A3IBEVCTXKNOH,"Kathy A. Welch ""katwel""",1,1,5,1331596800,Favorite Training and reward treat,These are the BEST treats for training and rew...


### I would like to see how many product get Score 5.

In [15]:
pd.read_sql("""SELECT COUNT(Score) FROM Reviews
             WHERE Score>4 ORDER BY Score DESC ;""", conn)

,COUNT(Score)
0,363122


### What is the average score from reviewers for the food?

In [17]:
pd.read_sql("""SELECT AVG(Score) FROM Reviews ;""", conn)

,AVG(Score)
0,4.183199


In [19]:
# Select review where the score is not 3 for the specific user ID to see if we have duplicated data
pd.read_sql("""
SELECT * FROM Reviews
WHERE Score != 3 AND UserId="AR5J8UI46CURR" ORDER BY ProductID
""", conn)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


- You can see that the same user has multiple reviews with same values for HelpfulnessNumerator, HelpfulnessDenominator, Score, Time, Summary and Text.
- Often our results will include many duplicate values. If we want to select all the unique values from a column, we use the DISTINCT keyword 

In [25]:
pd.read_sql(""" SELECT DISTINCT Summary, ProfileName FROM Reviews
WHERE Score=5 LIMIT 10;""", conn)

,Summary,ProfileName
0,Good Quality Dog Food,delmartian
1,Great taffy,"Michael D. Bigham ""M. Wassir"""
2,Great! Just as good as the expensive brands!,David C. Sullivan
3,"Wonderful, tasty taffy",Pamela G. Williams
4,Yay Barley,R. James
5,Healthy Dog Food,Carol A. Reed
6,The Best Hot Sauce in the World,Canadian Fan
7,"My cats LOVE this ""diet"" food better than thei...","A Poeng ""SparkyGoHome"""
8,Strawberry Twizzlers - Yummy,"Lynrie ""Oh HELL no"""
9,"Lots of twizzlers, just what you expect.",Brian A. Lee


## Obtaining Basic Information About DataFrame

###